# How to get MOC data from VESPA in Python

Illustrate search for TAP MOC for vespa using 'pyvo' and 'astropy'

## Import required modules

In [1]:
import pyvo as vo
from mpl_toolkits.basemap import Basemap

## Search a specific TAP service

We are looking for VVEx - VIRTIS/Venus Express database since it has a MOC

### Search the database containing this service

In [2]:
keywords = ["Venus"]

In [3]:
databasesList = vo.registry.search(servicetype='tap',keywords=keywords, includeaux=True)

In [4]:
for d in databasesList :
    print(d.short_name)

J/A+A/537/L2
J/A+A/585/A53
J/A+A/606/A53
J/A+A/613/A25
J/A+A/619/A99
J/A+A/627/A82
J/A+A/653/A41
J/AJ/154/207
J/AJ/158/196
J/AJ/160/131
J/AJ/160/259
J/AJ/162/216
J/ApJ/800/135
J/ApJ/807/45
J/ApJ/893/L27
J/MNRAS/462/4300
J/MNRAS/485/3999
J/MNRAS/519/6210
VI/127

GRB/IPN
KONUS
Cassini RPWS
BDIP
bdip.observatori
HST Planeto
VVEx
MESS-MAG-VSO
MESS-MAG-VSO-AVG
PVO-PHEM-ASC
PVO-PHEM-BIN
VEX-ASPERA4-DER
VEX-ASPERA4-ENG
VEX-ASPERA4-RAW
VEX-HIGH-MAG


### Search the services in the databases that were found

In [5]:
urlsList = set(databasesList.to_table().field("access_urls"))

In [6]:
tables = []

for url in urlsList :
    print("Searching in service " + url)
    service = vo.dal.TAPService(url)
    searchResult = service.search("SELECT schema_name, table_name FROM tap_schema.tables WHERE schema_name LIKE '%"+keywords[0]+"%' OR table_name LIKE '%"+keywords[0]+"%' OR description LIKE '%"+keywords[0]+"%'")
    for tableFound in searchResult["table_name"] :
        tables.append({"url":url,"name":tableFound})
print("DONE")

Searching in service https://vo-pds-ppi.igpp.ucla.edu/tap
Searching in service http://voparis-tap-planeto.obspm.fr/tap
Searching in service http://voparis-tap-maser.obspm.fr/tap
Searching in service http://tapvizier.cds.unistra.fr/TAPVizieR/tap
Searching in service http://vo.lmd.jussieu.fr/tap
Searching in service https://heasarc.gsfc.nasa.gov/xamin/vo/tap
DONE


In [7]:
for table in tables :
    print(table["name"]," at ", table["url"])

vex_cleaned_high_res_mag.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
vex_aspera4_els_raw.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
mess_mag_calibrated_vso.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
vex_aspera4_els_derived.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
mess_mag_calibrated_vso_avg.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
pvo_ephem_vso_asc.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
vex_aspera4_els_eng.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
pvo_ephem_vso_bin.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
vvex.epn_core  at  http://voparis-tap-planeto.obspm.fr/tap
J/A+A/627/A82/tablea1  at  http://tapvizier.cds.unistra.fr/TAPVizieR/tap
J/A+A/650/A156/tablec1  at  http://tapvizier.cds.unistra.fr/TAPVizieR/tap
J/A+A/627/A82/tablea3  at  http://tapvizier.cds.unistra.fr/TAPVizieR/tap
J/A+A/650/A156/table2  at  http://tapvizier.cds.unistra.fr/TAPVizieR/tap
J/A+A/537/L2/tablea1  at  http://tapvizier.cds.unistra.fr/TAPVi

## Only keep tables with a 'coverage' key (and so maybe a MOC)

In [8]:
def doContainMOC(table) :
    service = vo.dal.TAPService(table["url"])
    try :
        searchResult = service.search("SELECT TOP 1 * FROM "+table["name"]+" WHERE coverage IS NOT NULL")
        if searchResult == "" :
            return False
        else :
            return True
    except :
        return False

In [9]:
tablesWithMOC = []
for table in tables :
    print("Checking table ",table["name"])
    if doContainMOC(table) :
        tablesWithMOC.append(table)

Checking table  vex_cleaned_high_res_mag.epn_core
Checking table  vex_aspera4_els_raw.epn_core
Checking table  mess_mag_calibrated_vso.epn_core
Checking table  vex_aspera4_els_derived.epn_core
Checking table  mess_mag_calibrated_vso_avg.epn_core
Checking table  pvo_ephem_vso_asc.epn_core
Checking table  vex_aspera4_els_eng.epn_core
Checking table  pvo_ephem_vso_bin.epn_core
Checking table  vvex.epn_core
Checking table  J/A+A/627/A82/tablea1
Checking table  J/A+A/650/A156/tablec1
Checking table  J/A+A/627/A82/tablea3
Checking table  J/A+A/650/A156/table2
Checking table  J/A+A/537/L2/tablea1
Checking table  J/A+A/627/A82/tablea2
Checking table  J/AJ/162/216/fig11


In [10]:
for table in tablesWithMOC :
    print(table["name"])

vvex.epn_core


## Display the first 10 elements in the table
Here we only consider the first table found in the previous section

In [15]:
def getData(tapService, serviceStructure, query) :
    if doContainMOC(serviceStructure) :
        return tapService.search(query).to_table()
    else :
        return "NO MOC DATA"

In [16]:
table = tablesWithMOC[0]
tapService = vo.dal.TAPService(table["url"])

data = tapService.search("""SELECT TOP 10 * FROM """ + table["name"]).to_table()
print("\nAll data: ",data)


All data:  granule_uid granule_gid   obs_id  ... science_case_id sc_pointing_mode
                                  ...                                 
----------- ----------- --------- ... --------------- ----------------
 VI0478_03C  calibrated VI0478_03 ...               3           MOSAIC
 VI0478_03G    geometry VI0478_03 ...               3           MOSAIC
 VI0478_04C  calibrated VI0478_04 ...               3           MOSAIC
 VI0478_04G    geometry VI0478_04 ...               3           MOSAIC
 VI0478_05C  calibrated VI0478_05 ...               3           MOSAIC
 VI0478_05G    geometry VI0478_05 ...               3           MOSAIC
 VI0478_06C  calibrated VI0478_06 ...               3           MOSAIC
 VI0478_06G    geometry VI0478_06 ...               3           MOSAIC
 VI0478_07C  calibrated VI0478_07 ...               3           MOSAIC
 VI0478_07G    geometry VI0478_07 ...               3           MOSAIC


## ...


In [17]:
import astropy.units as u
from astropy.coordinates import SkyCoord

from astroquery.cds import cds
from astroquery.vizier import Vizier

from mocpy import MOC

from ipyaladin import Aladin

In [35]:
aladin = Aladin(target=data[0]["granule_uid"])
aladin

Aladin(options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey', 'overlay_sur…

In [36]:
aladin.add_table(data)

In [27]:
mocData = tapService.search("""SELECT TOP 1 coverage FROM """ + table["name"]).to_table()[0]["coverage"]

In [34]:
moc = MOC.from_str(mocData)
jsonMoc = moc.serialize(format='json', optional_kw_dict=None, pre_v2=False)
aladin.add_moc_from_dict(jsonMoc, {"adaptativeDisplay": False})

In [37]:
#TODO: Load the corresponding survey (Venus for instance) + fix "target"